# Explore here

In [35]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [4]:
#Importo los documentos csv que contiene la data
test_data = pd.read_csv(r"C:\Users\macar\OneDrive\Escritorio\REPOS_2\Proyecto-rbol-de-Decisi-n\data\processed\clean_test_diabetes.csv")
train_data = pd.read_csv(r"C:\Users\macar\OneDrive\Escritorio\REPOS_2\Proyecto-rbol-de-Decisi-n\data\processed\clean_train_diabetes.csv")

In [5]:
#Defino las muestras de prueba y de entrenamiento
X_train = train_data.drop(["Outcome"], axis = 1)
y_train = train_data["Outcome"]

X_test = test_data.drop(["Outcome"], axis = 1)
y_test = test_data["Outcome"]

## **Boosting por gradiente**

sklearn

In [42]:
from sklearn.ensemble import GradientBoostingClassifier

model_1 = GradientBoostingClassifier(random_state = 42)

model_1.fit(X_train, y_train)

y_pred_train = model_1.predict(X_train)
y_pred_test = model_1.predict(X_test)

In [43]:
from función import get_metrics

get_metrics(y_train, y_test, y_pred_test= y_pred_test , y_pred_train= y_pred_train)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.938111,0.905941,0.919602,0.958115,0.859155,0.980050
Test,0.746753,0.654867,0.730303,0.637931,0.672727,0.787879
Diferencia,0.191358,0.251073,0.189299,0.320184,0.186428,0.192171


XGBoost

In [39]:
from xgboost import XGBClassifier

model_2 = XGBClassifier(random_state = 42)

model_2.fit(X_train, y_train)

y_pred_train = model_2.predict(X_train)
y_pred_test_1 = model_2.predict(X_test)

In [10]:
get_metrics(y_train, y_test, y_pred_test= y_pred_test_1 , y_pred_train= y_pred_train)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
Test,0.707792,0.615385,0.69596,0.580645,0.654545,0.737374
Diferencia,0.292208,0.384615,0.30404,0.419355,0.345455,0.262626


A primera vista, con los dos algortimos es sus predeterminados parámetros, podemos ver que el algoritmo boosting de sklearn no sobrejusta tanto el modelo, y por lo tanto puede generalizar más.

## Optimización del modelo

Para la optimización de los dos modelos, voy a crear dos submodelos para cada uno. Un submodelo será complejo, es decir, tendrá más posibilidad de overfitting y otro más senzillo. Para ello voy a definir los hiperparámetros de forma que cumplan la función para el tipo de modelo que se crea.

Para Gradient Boosting

In [21]:
model_1_complex = GradientBoostingClassifier(n_estimators=500, learning_rate= 0.01, loss='log_loss', subsample=0.8, max_depth=10, min_samples_split=10, min_samples_leaf=4, max_features= 4)
model_1_easy = GradientBoostingClassifier(n_estimators=100, learning_rate= 0.1, loss='log_loss', subsample=1, max_depth=3, min_samples_split=2, min_samples_leaf=1, max_features= 4 )

In [22]:
model_1_complex.fit(X_train,y_train)
model_1_easy.fit(X_train, y_train)

GradientBoostingClassifier(max_features=4, subsample=1)

In [23]:
y_pred_train_m1c = model_1_complex.predict(X_train)
y_pred_test_m1c = model_1_complex.predict(X_test)

get_metrics(y_train, y_test, y_pred_test= y_pred_test_m1c , y_pred_train= y_pred_train_m1c)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Test,0.727273,0.637931,0.715152,0.606557,0.672727,0.757576
Diferencia,0.272727,0.362069,0.284848,0.393443,0.327273,0.242424


In [24]:
y_pred_train_m1e = model_1_easy.predict(X_train)
y_pred_test_m1e = model_1_easy.predict(X_test)

get_metrics(y_train, y_test, y_pred_test= y_pred_test_m1e , y_pred_train= y_pred_train_m1e)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.921824,0.880597,0.900530,0.936508,0.830986,0.970075
Test,0.720779,0.626087,0.706061,0.600000,0.654545,0.757576
Diferencia,0.201045,0.254510,0.194470,0.336508,0.176440,0.212499


Observando los resultados se observa que el modelo complejo de Gradient Boosting se sobreajusta al 100% y obtiene unos resultados más pobres en el conjunto de datos que el modelo sin optimizar. Por otra parte, el modelo sencillo aunque no sobreajusta tanto los datos de entrenamiento, no mejora el resultado obtenido anteriormente.

Para XGBoosting

In [25]:
model_2_complex = XGBClassifier(n_estimators=500, learning_rate=0.01, objective='binary:logistic', subsample=0.8, max_depth=10, gamma=0.5, min_child_weight=3, colsample_bytree=0.8)
model_2_easy =  XGBClassifier(n_estimators=100, learning_rate=0.1, objective='binary:logistic', subsample=1, max_depth=3, gamma=0, min_child_weight=1, colsample_bytree=1)

In [27]:
model_2_complex.fit(X_train,y_train)
model_2_easy.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [28]:
y_pred_train_m2c = model_2_complex.predict(X_train)
y_pred_test_m2c = model_2_complex.predict(X_test)

get_metrics(y_train, y_test, y_pred_test= y_pred_test_m2c , y_pred_train= y_pred_train_m2c)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.949511,0.925659,0.939336,0.946078,0.906103,0.972569
Test,0.759740,0.678261,0.748485,0.650000,0.709091,0.787879
Diferencia,0.189771,0.247399,0.190851,0.296078,0.197012,0.184690


In [30]:
y_pred_train_m2e = model_2_easy.predict(X_train)
y_pred_test_m2e = model_2_easy.predict(X_test)

get_metrics(y_train, y_test, y_pred_test= y_pred_test_m2e , y_pred_train= y_pred_train_m2e)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.907166,0.859951,0.887107,0.902062,0.821596,0.952618
Test,0.733766,0.643478,0.720202,0.616667,0.672727,0.767677
Diferencia,0.173400,0.216473,0.166905,0.285395,0.148869,0.184942


En comparación a GradienBoosting, retocando los parametros de XGBoosting, hemos obtenido un resultado en el modelo complejo superior al modelo de XGBoost sin optimizar, incluso superando al modelo GradiendBoosting.

### Comparación con los modelos RandomForest y DecisionTree

In [32]:
from pickle import load

# Cargar el modelo RandomForest guardado
random_forest_model = load(open(r"C:\Users\macar\OneDrive\Escritorio\REPOS_2\Random-Forest-\models\random_forest_crit-entro_maxdepth-10_min_samples_leaf=5, min_samples_split=15, rs = 42.sav", "rb"))
# Cargar el modelo RandomForest guardado
decision_tree_model = load(open(r"C:\Users\macar\OneDrive\Escritorio\REPOS_2\Proyecto-rbol-de-Decisi-n\models\tree_classifier_crit-entro_maxdepth-10_maxfeat-4_mins_leaf-20_42.sav", "rb"))


In [33]:
#Aqui defino las y_pred para los dos modelos importados
y_pred_rf = random_forest_model.predict(X_test)
y_pred_dt = decision_tree_model.predict(X_test)

Voy a comparar los cuatro modelos escogiendo el mejor resultado obtenido en la métrica "accuracy"

In [44]:
# Accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
accuracy_gb = accuracy_score(y_test, y_pred_test)
accuracy_xgb = accuracy_score(y_test, y_pred_test_m2c )

print(f"Accuracy RandomForest: {accuracy_rf}")
print(f"Accuracy DecisionTree: {accuracy_dt}")
print(f"Accuracy GradientBoosting: {accuracy_gb}")
print(f"Accuracy XGBoosting: {accuracy_xgb}")

Accuracy RandomForest: 0.7532467532467533
Accuracy DecisionTree: 0.7597402597402597
Accuracy GradientBoosting: 0.7467532467532467
Accuracy XGBoosting: 0.7597402597402597


Según los resultados obtenidos vemos que hay un empate entre el modelo de DecisionTree y el modelo de XGBoosting, siendo el modelo de GradientBoosting el que menos accuracy ha otenido de los 4.

In [45]:
#Almaceno el modelo
from pickle import dump

dump(model_2_complex, open("../models/xgbboost,n_estimators=500, learning_rate=0.01, objective='binary:logistic', subsample=0.8, max_depth=10, gamma=0.5, min_child_weight=3, colsample_bytree=0.8.sav", "wb"))

## Comentario Final:
Para la optimización de los modelos de GradienBoosting y XGBoosting he probado de hacer GridSearch para encontrar la mejor configuración de los hiperparámetros, pero cuando le he dado a run el output no salía en mucho tiempo (más de 30 min), por lo que he decidido configurarlo yo mismo haciendo 4 modelos en total, 2 de complejos y 2 de sencillos.